In [0]:
%sql

WITH parsed_documents AS (
    SELECT
      path,
      ai_parse_document(
        content,
        map(
          'imageOutputPath', '/Volumes/udemy_db_ws/invoices/parsed_images/',
          'descriptionElementTypes', '*'
        )
      ) AS parsed
    FROM READ_FILES('/Volumes/udemy_db_ws/invoices/shopping_invoices/*.{pdf,jpg,jpeg,png,doc,docx,ppt,pptx}', format => 'binaryFile')
  ),
  parsed_text AS (
    SELECT
      path,
      concat_ws(
        '\n\n',
        transform(
          try_cast(parsed:document:elements AS ARRAY<VARIANT>),
          element -> try_cast(element:content AS STRING)
        )
      ) AS text
    FROM parsed_documents
    WHERE try_cast(parsed:error_status AS STRING) IS NULL
  )
  SELECT
    path,
    text,
    ai_query(
      'databricks-claude-sonnet-4',
      concat(
        'Extract vendor name, date, invoice number, and items purchased from this document. ',
        'Return the result as a JSON object with keys: vendor, date, invoice_number, items (as an array). ',
        text
      ),
      returnType => 'STRING'
    ) AS structured_data
  FROM parsed_text
  WHERE text IS NOT NULL;